In [ ]:
import os
import pickle
import jsonlines
import pandas as pd
import numpy as np
import json
import copy
from tqdm import tqdm
from collections import defaultdict
import requests

In [ ]:
id_map = json.load(open("./handled/id_map.json"))
item_dict = json.load(open("./handled/item2attributes.json", "r"))

In [ ]:
def load_dataset():
        '''Load train, validation, test dataset'''

        usernum = 0
        itemnum = 0
        User = defaultdict(list)    # default value is a blank list
        user_train = {}
        user_valid = {}
        user_test = {}
        # assume user/item index starting from 1
        f = open('./handled/inter.txt', 'r')
        for line in f:  # use a dict to save all seqeuces of each user
            u, i = line.rstrip().split(' ')
            u = int(u)
            i = int(i)
            usernum = max(u, usernum)
            itemnum = max(i, itemnum)
            User[u].append(i)

        for user in tqdm(User):
            nfeedback = len(User[user])
            #nfeedback = len(User[user])
            if nfeedback < 3:
                user_train[user] = User[user]
                user_valid[user] = []
                user_test[user] = []
            else:
                user_train[user] = User[user][:-2]
                user_valid[user] = []
                user_valid[user].append(User[user][-2])
                user_test[user] = []
                user_test[user].append(User[user][-1])
        
        return user_train
        

In [ ]:
inter = load_dataset()

In [ ]:
prompt_template = "The user has visited following point of interests: \n<HISTORY> \nplease conclude the user's perference."

In [ ]:
def get_user_prompt(history):

    user_str = copy.deepcopy(prompt_template)
    hist_str = ""
    for item in history:
        item_str = item_dict[id_map["id2item"][str(item)]]["name"]
        hist_str = hist_str + item_str + ", "

    # limit the prompt length
    if len(hist_str) > 8000:
        hist_str = hist_str[-8000:]

    user_str = user_str.replace("<HISTORY>", hist_str)

    return user_str

In [ ]:
user_data = {}

for user, history in tqdm(inter.items()):
    user_data[user] = get_user_prompt(history)

In [ ]:
json.dump(user_data, open("./handled/user_str.json", "w"))

In [ ]:
user_data = json.load(open("./handled/user_str.json", "r"))

In [ ]:
url = ""

payload = json.dumps({
   "model": "text-embedding-ada-002",
   "input": "The food was delicious and the waiter..."
})
headers = {
   'Authorization': '',
   'User-Agent': 'Apifox/1.0.0 (https://apifox.com)',
   'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

In [ ]:
def get_response(prompt):
    url = ""

    payload = json.dumps({
    "model": "text-embedding-ada-002",
    "input": prompt
    })
    headers = {
    'Authorization': '',
    'User-Agent': 'Apifox/1.0.0 (https://apifox.com)',
    'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    re_json = json.loads(response.text)

    return re_json["data"][0]["embedding"]

In [ ]:
user_emb = {}

In [ ]:
while 1:    # avoid broken due to internet connection
    try:
        for key, value in tqdm(user_data.items()):
            if key not in user_emb.keys():
                user_emb[key] = get_response(value)
    except:
        continue
    if len(user_emb) == len(user_data):
        break

In [ ]:
len(user_emb)

In [ ]:
emb_list = []
for key, value in tqdm(user_emb.items()):
    emb_list.append(value)

emb_list = np.array(emb_list)
pickle.dump(emb_list, open("./handled/usr_emb_np.pkl", "wb"))